In [19]:
from sklearn.metrics import confusion_matrix
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import seaborn as sns

%matplotlib inline


plt.rcParams["font.family"] = "Palatino"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

label_size = 22 + 2
tick_size = 20 + 2
title_size = 24 + 2
legend_size = 22 + 2

In [20]:
experiments = [
    'lt-at',
    'lt-ct',
    'lt-al',
    'lt-ca',
    'at-ct',
    'at-al',
    'at-ca'
]
al = set(range(0, 24))
lt = set(range(24, 63))
at = set(range(63, 89))
ca = set(range(89, 127))
ct = set(range(127, 147))

cluster_name_to_cluster_id = {
    'lt': 0,
    'at': 1,
    'al': 2,
    'ca': 3,
    'ct': 4
}

clusters = [lt, at, al, ca, ct]
cluster_str = ['lt', 'at', 'al', 'ca', 'ct']
image_id_to_cluster_id = {}
image_id_to_cluster_str = {}
for cluster_id, cluster in enumerate(clusters):
    for image_id in cluster:
        image_id_to_cluster_id[image_id] = cluster_id
        image_id_to_cluster_str[image_id] = cluster_str[cluster_id]

In [21]:
np.random.seed(42)

for exp_idx, exp in enumerate(experiments):
    M = np.zeros((5, 5))

    y_true = []
    y_pred = []
    counter = defaultdict(list)

    with open(f"data/{exp}.json", "r") as fp:
        data = json.load(fp)
        for worker in data:
            for answer in worker["answer"]:
                if "feedback" in answer or answer["questionNum"] == 0:
                    continue
                worker_ans = 0 if "NOT" in answer["clusters"] else 1
                image_a = answer["originalOrder"][0]["imageId"]
                image_b = answer["originalOrder"][1]["imageId"]
                counter[
                    (image_id_to_cluster_id[image_a], image_id_to_cluster_id[image_b])
                ].append(len(y_true))
                true_ans = int(
                    image_id_to_cluster_id[image_a] == image_id_to_cluster_id[image_b]
                )
                M[
                    image_id_to_cluster_id[image_a], image_id_to_cluster_id[image_b]
                ] += worker_ans
                y_true.append(true_ans)
                y_pred.append(worker_ans)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    cluster_ids = list(map(lambda x: cluster_name_to_cluster_id[x], exp.split("-")))
    M = M[cluster_ids][:, cluster_ids]

    a = y_pred[y_true == 0]
    std = np.sqrt(np.mean((a - np.mean(a)) ** 2) / 300)
    print(f'"{exp}": {std},')

    # print(
    #     f"{exp} & {y_pred[y_true == 0].mean():.3f} $\\pm$ {std:.3f} \\\\"
    # )

"lt-at": 0.028396009109262755,
"lt-ct": 0.02855533938526046,
"lt-al": 0.018761663039293715,
"lt-ca": 0.013711309200802092,
"at-ct": 0.0244191788620399,
"at-al": 0.023515952032609696,
"at-ca": 0.016795061002392163,


In [22]:
np.sum(y_true == 0)

300